In [4]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks

In [5]:
# Define the dataset directory (update this path to where your Places365 dataset is stored)
DATASET_PATH = "D:/Music_v3/Emotion_Based_Music_Player_with_Age_Preference/places365_standard"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Define dataset paths (assuming you have separate folders for training and validation)
TRAIN_PATH = os.path.join(DATASET_PATH, "train")
VAL_PATH = os.path.join(DATASET_PATH, "val")


In [6]:
# Load the full training dataset from the train directory
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PATH,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

# Load the full validation dataset from the val directory
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_PATH,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

Found 1675023 files belonging to 340 classes.
Found 36500 files belonging to 365 classes.


In [7]:
# # Optimize dataset performance on CPU: cache and prefetch
# AUTOTUNE = tf.data.AUTOTUNE
# train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
# Get class names (scene categories) from the training dataset
scene_classes = train_ds.class_names
num_classes = len(scene_classes)
print("Scene Categories:", scene_classes)


Scene Categories: ['airfield', 'airplane_cabin', 'airport_terminal', 'alcove', 'alley', 'amphitheater', 'amusement_arcade', 'amusement_park', 'apartment_building-outdoor', 'aquarium', 'aqueduct', 'arcade', 'arch', 'archaelogical_excavation', 'archive', 'arena-performance', 'arena-rodeo', 'army_base', 'art_gallery', 'art_school', 'art_studio', 'artists_loft', 'assembly_line', 'athletic_field-outdoor', 'atrium-public', 'attic', 'auditorium', 'auto_factory', 'auto_showroom', 'balcony-exterior', 'ball_pit', 'ballroom', 'bamboo_forest', 'banquet_hall', 'bar', 'barn', 'barndoor', 'baseball_field', 'basketball_court-indoor', 'bathroom', 'bazaar-indoor', 'bazaar-outdoor', 'beach', 'beach_house', 'beauty_salon', 'bedchamber', 'bedroom', 'beer_hall', 'berth', 'biology_laboratory', 'boardwalk', 'boat_deck', 'boathouse', 'bookstore', 'booth-indoor', 'bow_window-indoor', 'bowling_alley', 'boxing_ring', 'bridge', 'building_facade', 'bullring', 'burial_chamber', 'bus_interior', 'bus_station-indoor', 

In [9]:
# Define the base model (MobileNetV2) for feature extraction
# This lightweight model is efficient for CPU-based training/inference.
base_model = keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model to use pre-trained features

# Build the complete classification model
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

In [10]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Define early stopping to halt training if validation loss stops improving
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [12]:
# Train the model using the full dataset
EPOCHS = 10
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=EPOCHS,
                    callbacks=[early_stopping])

Epoch 1/10
   23/52345 ━━━━━━━━━━━━━━━━━━━━ 5:18:09 365ms/step - accuracy: 0.0070 - loss: 5.6578

KeyboardInterrupt: 